# **Database Optimization Techniques**

Database often becomes the bottleneck in software performance. Having an optimized database is essential for high performing systems.

# Indexing:
**Create Indexes:**

Indexes are data structures that provide a quick lookup mechanism, significantly improving query performance. They work by creating a sorted data structure that allows the database engine to quickly locate the rows that satisfy a WHERE clause.


In [ ]:
CREATE INDEX idx_username ON users(username);

**Use Composite Indexes:**

Composite indexes involve multiple columns and are useful for queries that filter or sort based on multiple conditions. This reduces the need for separate indexes on each column and improves the efficiency of the query planner.

In [ ]:
CREATE INDEX idx_name_age ON employees(name, age);

# Normalization and Denormalization:
**Normalization:**

This process organizes data to minimize redundancy and dependency, reducing the likelihood of data anomalies. By breaking down large tables into smaller, related ones, normalization ensures data consistency. However, it may lead to more complex queries.

SQL Code Snippet (Example of Third Normal Form):

In [ ]:
CREATE TABLE customers (
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(100),
    address VARCHAR(255)
);

CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

**Denormalization:**

While normalization reduces redundancy, denormalization introduces controlled redundancy to improve query performance, especially for read-heavy operations. This can involve adding redundant columns or tables strategically.

In [ ]:
CREATE TABLE denormalized_orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    customer_name VARCHAR(100),
    order_date DATE
);

# Query Optimization
**Optimize Queries:**

Regularly analyze and optimize frequently used queries. Use tools like EXPLAIN to understand the query execution plan and identify areas for improvement.

In [ ]:
EXPLAIN SELECT * FROM orders WHERE customer_id = 123;

# Avoid SELECT *:

Retrieve only the necessary columns rather than selecting all columns. This reduces the amount of data transferred and processed, improving query performance.

In [ ]:
SELECT order_id, order_date FROM orders WHERE customer_id = 123;

# Partitioning
**Partition Tables:**

Partitioning involves dividing large tables into smaller, more manageable pieces. This can significantly improve query performance by allowing the database engine to work on smaller subsets of data, leading to faster query execution.

In [ ]:
CREATE TABLE sales (
    sale_id INT PRIMARY KEY,
    sale_date DATE,
    amount DECIMAL(10, 2)
) PARTITION BY RANGE (YEAR(sale_date)) (
    PARTITION p0 VALUES LESS THAN (1990),
    PARTITION p1 VALUES LESS THAN (2000),
    PARTITION p2 VALUES LESS THAN (2010),
    PARTITION p3 VALUES LESS THAN (2020),
    PARTITION p4 VALUES LESS THAN (MAXVALUE)
);

**Partition Pruning:**

Ensure that the query planner prunes unnecessary partitions during query execution. This prevents scanning the entire dataset and improves performance.

In [ ]:
SELECT * FROM sales WHERE sale_date >= '2022-01-01' AND sale_date < '2023-01-01';

# Caching
**Query Caching:**

Implement a caching mechanism to store the results of frequently executed queries. This reduces the load on the database by serving cached results, enhancing response times.

In [ ]:
-- Pseudocode
DECLARE @cacheKey NVARCHAR(255) = 'query_cache_key';
DECLARE @cachedResult NVARCHAR(MAX);

SET @cachedResult = REDIS.GET(@cacheKey);

IF @cachedResult IS NULL
BEGIN
    -- Execute the query and store the result in the cache
    SET @cachedResult = EXECUTE_QUERY('SELECT * FROM large_table');
    REDIS.SET(@cacheKey, @cachedResult, EXPIRY_TIME);
END

-- Use @cachedResult for further processing

**Object Caching:**

Cache frequently accessed objects or data in the application layer to minimize database queries. This can be achieved using in-memory caching libraries or frameworks.

In [ ]:
from django.core.cache import cache

def get_user_data(user_id):
    # Try to fetch user data from cache
    user_data = cache.get(f'user_{user_id}')

    if user_data is None:
        # If not in cache, fetch from the database
        user_data = User.objects.get(id=user_id)

        # Store the data in cache for future requests
        cache.set(f'user_{user_id}', user_data, TIMEOUT)

    return user_data

# Regular Maintenance:
**Update Statistics:**

Keeping statistics up-to-date is crucial for the query planner to make informed decisions about execution plans. Regularly update statistics to ensure accurate and efficient query optimization.

In [ ]:
-- Update statistics for a table
UPDATE STATISTICS table_name;

**Data Archiving:**

Archive or purge old data that is no longer needed. This can improve query performance and reduce storage requirements, especially in systems with large historical datasets.

In [ ]:
-- Archive data older than a certain date
DELETE FROM historical_data WHERE date < '2020-01-01';

# Hardware Optimization:
**Optimize Server Configuration:**

Adjust database server settings and configurations based on the workload and hardware capabilities. This includes parameters such as buffer sizes, cache settings, and connection limits.

In [ ]:
-- Example: Increase the size of the query cache
SET GLOBAL query_cache_size = 256M;

**Use SSDs:**

Consider using Solid State Drives (SSDs) for storage. SSDs provide faster data access compared to traditional Hard Disk Drives (HDDs), resulting in improved overall database performance.

# Concurrency Control:
**Isolation Levels:**

Adjust isolation levels based on the requirements of your application. Isolation levels control the visibility of changes made by one transaction to other transactions. Choosing the appropriate isolation level is crucial for balancing consistency and performance.

In [ ]:
-- Set isolation level to READ COMMITTED
SET TRANSACTION ISOLATION LEVEL READ COMMITTED;

# Connection Pooling:
**Use Connection Pooling:**

Reuse database connections to avoid the overhead of establishing new connections for each request. Connection pooling helps manage and reuse database connections efficiently.

In [ ]:
HikariConfig config = new HikariConfig();
config.setJdbcUrl("jdbc:mysql://localhost:3306/database");
config.setUsername("username");
config.setPassword("password");
config.setMaximumPoolSize(10);

# Database Design:
**Efficient Schema Design:**

Design the database schema with performance in mind. Optimize data types, use appropriate constraints, and minimize unnecessary relationships. A well-designed schema can significantly impact query efficiency.

In [ ]:
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(255),
    price DECIMAL(10, 2),
    -- Additional columns as needed
);

# Monitoring and Profiling:
**Regular Monitoring:**

Implement monitoring tools to track database performance over time. Regularly monitor key metrics such as CPU usage, memory usage, and query execution times to identify potential issues.

In [ ]:
SHOW STATUS LIKE 'cpu%';

**Profiling Queries:**

Profile and analyze the performance of individual queries to pinpoint bottlenecks. Tools like the MySQL Performance Schema can provide detailed insights into query execution.

In [ ]:
-- Enable Performance Schema
SET GLOBAL performance_schema = ON;

-- Profile a specific query
SELECT * FROM orders WHERE customer_id = 123;

# 1. Indeksite kasutamine
Indeksid kiirendavad päringute töötlemist, luues tabelisse lisastruktuuri, mis teeb otsingu kiiremaks.

Näide: Kujutame ette, et meil on tabel Employees, kus on palju ridu. Kui tahame leida töötajat nime järgi, oleks mõistlik luua indeks veerule name:

In [ ]:
CREATE INDEX idx_name ON Employees(name);

Nüüd, kui sooritad järgneva päringu,
töötleb andmebaas seda palju kiiremini, kuna see kasutab eelnevalt loodud indeksit.

In [ ]:
SELECT * FROM Employees WHERE name = 'John Doe';

# 2. **Vältige SELECT ***
SELECT * pärib kõik veerud tabelist, mis võib olla väga ressursimahukas, eriti kui enamikku andmeid ei ole vaja. Parem on valida ainult vajalikud veerud.

Näide:

In [ ]:
-- Halvem variant
SELECT * FROM Employees;

-- Parem variant
SELECT name, position FROM Employees;

# 3. Väljade normaliseerimine
Normaliseerimine tähendab andmete jaotamist mitmesse tabelisse, et vähendada andmete kordumist ja muuta andmebaas tõhusamaks.

Näide: Kui teil on tabel Orders, kus iga tellimus sisaldab ka kliendi andmeid, siis võib olla mõistlik viia kliendi andmed eraldi tabelisse Customers ja viidata Orders tabelis Customers tabeli ID-le.

In [ ]:
-- Ebaefektiivne struktuur
Orders (order_id, customer_name, customer_address, ...)

-- Parem struktuur
Customers (customer_id, customer_name, customer_address, ...)
Orders (order_id, customer_id, ...)

# 4. JOIN-i efektiivne kasutamine
JOIN-i kasutamine võib olla kulukas, eriti kui seotud tabelid on suured. Oluline on kasutada õiget tüüpi JOIN-i ja tagada, et JOIN-tingimused kasutavad indekseid.

Näide: Kui liidad suurt Orders tabelit väiksema Customers tabeliga:

In [ ]:
-- Halvem variant
SELECT * FROM Orders, Customers WHERE Orders.customer_id = Customers.customer_id;

-- Parem variant
SELECT Orders.order_id, Customers.customer_name
FROM Orders
JOIN Customers ON Orders.customer_id = Customers.customer_id;

# 5. WHERE tingimuste optimeerimine
Tingimuste kasutamine WHERE lauses võib olla märkimisväärne jõudluse parendaja, kui kasutatakse indekseid ja vältitakse keerulisi funktsioone.

Näide:

In [ ]:
-- Halvem variant (väldib indeksite kasutamist, kui `date` veerul on indeks)
SELECT * FROM Orders WHERE YEAR(order_date) = 2023;

-- Parem variant (kasutab indekseid)
SELECT * FROM Orders WHERE order_date BETWEEN '2023-01-01' AND '2023-12-31';

# 6. Liiga paljude JOIN-ide ja ALAMpäringute vältimine
Suured päringud, mis kasutavad palju JOIN-e ja alampäringuid, võivad muutuda aeglaseks. Mõnikord on parem jagada päringud mitmeks väiksemaks päringuks.

Näide: Kui päringus on palju JOIN-e, kaalu päringu jaotamist väiksemateks osadeks või kasutades ajutisi tabeleid:

In [ ]:
-- Alternatiiv suurele JOIN päringule
CREATE TEMPORARY TABLE TempOrders AS
SELECT * FROM Orders WHERE order_date BETWEEN '2023-01-01' AND '2023-12-31';

SELECT * FROM TempOrders JOIN Customers ON TempOrders.customer_id = Customers.customer_id;

# 7. Vaadete (Views) kasutamine keerukate päringute puhul
Kui sul on keerukaid päringuid, mida kasutatakse sageli, võib olla kasulik luua neist vaade (view), mis teeb nende kasutamise lihtsamaks ja kiiremini töötavaks.

Näide:

In [ ]:
CREATE VIEW OrderSummary AS
SELECT customer_id, COUNT(order_id) AS order_count, SUM(order_amount) AS total_spent
FROM Orders
GROUP BY customer_id;

Nüüd saad vaadet kasutada lihtsates päringutes:

In [ ]:
SELECT * FROM OrderSummary WHERE total_spent > 1000;

# 8. Tabelite ja andmete arhiivimine
Suure mahuga tabelid võivad aeglustada päringuid. Vana või vähekasutatav info võiks arhiivida eraldi tabelitesse.

Näide:

In [ ]:
-- Liigutame vanad tellimused arhiivitabelisse
INSERT INTO ArchivedOrders SELECT * FROM Orders WHERE order_date < '2023-01-01';
DELETE FROM Orders WHERE order_date < '2023-01-01';

# 9. Andmebaasi ühenduste ja seansside optimeerimine
Paljude ühenduste haldamine võib koormata andmebaasi. Seetõttu tuleks ühendused ja seansid optimeerida, kasutades näiteks ühenduste taas- ja koondkasutamist.

# **Optimeerimise näited.**

# 1. GROUP BY optimeerimine

In [ ]:
-- Algne päring
SELECT department, COUNT(*)
FROM Employees
GROUP BY department;

In [ ]:
-- Optimeeritud päring
SELECT department, COUNT(employee_id)
FROM Employees
GROUP BY department;

Selgitus: COUNT(*) loeb kõik read, kuid COUNT konkreetse veeru järgi (näiteks employee_id) võib olla kiirem, eriti kui veerg on indekseeritud.

# 2. LIMIT ja ORDER BY kombinatsioon

In [ ]:
-- Algne päring:
SELECT * FROM Orders
ORDER BY order_date DESC;

In [ ]:
-- Optimeeritud päring:
SELECT * FROM Orders
ORDER BY order_date DESC
LIMIT 10;

Selgitus: Kui soovite ainult esimesi tulemusi, kasutage LIMIT, et vähendada tagastatavate ridade arvu ja kiirendada päringu töötlemist.

# 3. LIKE optimeerimine

In [ ]:
-- Algne päring:
SELECT * FROM Employees
WHERE name LIKE '%John%';

In [ ]:
-- Optimeeritud päring:
SELECT * FROM Employees
WHERE name LIKE 'John%';

Selgitus: Kui võimalik, vältige LIKE mustreid, mis algavad '%...', kuna need takistavad indeksi kasutamist. Otsing LIKE 'John%' suudab kasutada indekseid ja on palju kiirem.

# 4. JOIN tingimuse optimeerimine

In [ ]:
-- Algne päring:
SELECT Orders.order_id, Customers.name
FROM Orders, Customers
WHERE Orders.customer_id = Customers.customer_id
AND Orders.order_date > '2023-01-01';

In [ ]:
-- Optimeeritud päring:
SELECT Orders.order_id, Customers.name
FROM Orders
JOIN Customers ON Orders.customer_id = Customers.customer_id
WHERE Orders.order_date > '2023-01-01';

Selgitus: Kasutage alati JOIN sünteksit, mis on paremini loetav ja optimeerib päringu täitmist. SQL optimeerijad eelistavad seda vormi.

# 5. HAVING ja WHERE kasutamine

In [ ]:
-- Algne päring:
SELECT department, COUNT(*)
FROM Employees
GROUP BY department
HAVING COUNT(*) > 5;

In [ ]:
-- Optimeeritud päring:
SELECT department, COUNT(*)
FROM Employees
WHERE department IS NOT NULL
GROUP BY department
HAVING COUNT(*) > 5;

Selgitus: Kui võimalik, viige filtreerimine WHERE klausli kaudu läbi enne rühmitamist, et vähendada päringu töötlemiseks vajalikku andmehulka.

# 6. Päringute mittevajalike veergude eemaldamine

In [ ]:
-- Algne päring:
SELECT order_id, order_date, customer_id, product_id, quantity, price
FROM Orders
WHERE order_date BETWEEN '2023-01-01' AND '2023-12-31';

In [ ]:
-- Optimeeritud päring:
SELECT order_id, order_date, customer_id
FROM Orders
WHERE order_date BETWEEN '2023-01-01' AND '2023-12-31';

Selgitus: Valige ainult need veerud, mida tegelikult vajate. See vähendab andmete kogust, mida tuleb töödelda ja üle võrgu edastada.

# 7. Alampäringu (Subquery) optimeerimine

In [ ]:
-- Algne päring:
SELECT * FROM Employees
WHERE salary > (SELECT AVG(salary) FROM Employees);

In [ ]:
-- Optimeeritud päring:
SELECT e.*
FROM Employees e
JOIN (SELECT AVG(salary) AS avg_salary FROM Employees) AS subquery
ON e.salary > subquery.avg_salary;

Selgitus: Kasutades JOIN-i alampäringuga, mida täidetakse ainult üks kord, saate vähendada alampäringu mitmekordset täitmist ja kiirendada päringut.

# 8. DISTINCT vs. GROUP BY

In [ ]:
-- Algne päring:
SELECT DISTINCT department_id
FROM Employees;

In [ ]:
-- Optimeeritud päring:
SELECT department_id
FROM Employees
GROUP BY department_id;

Selgitus: Mõnel juhul töötab GROUP BY kiiremini kui DISTINCT, eriti suurte andmekogumite puhul.

# 9. Transaktsioonide optimeerimine

In [ ]:
-- Algne päring:
BEGIN;
UPDATE Orders SET status = 'Processed' WHERE order_id = 1;
UPDATE Inventory SET stock = stock - 1 WHERE product_id = 1;
COMMIT;

In [ ]:
-- Optimeeritud päring:
BEGIN;
UPDATE Orders SET status = 'Processed' WHERE order_id = 1;
UPDATE Inventory SET stock = stock - 1 WHERE product_id = 1;
COMMIT;

Selgitus: Kuigi need kaks päringut tunduvad samad, on oluline märkida, et transaktsioone tuleks kasutada ainult seal, kus need on vajalikud, ning need tuleks hoida võimalikult lühikesed. Optimeerimiseks saab kasutada ka indekseid, et kiirendada nende päringute täitmist transaktsioonide sees.

# 10. NOT IN vs NOT EXISTS

In [ ]:
-- Algne päring:
SELECT * FROM Employees
WHERE department_id NOT IN (SELECT department_id FROM Departments WHERE active = 0);

In [ ]:
-- Optimeeritud päring:
SELECT * FROM Employees e
WHERE NOT EXISTS (SELECT 1 FROM Departments d WHERE e.department_id = d.department_id AND d.active = 0);

Selgitus: NOT EXISTS on tihti efektiivsem kui NOT IN, kuna see lõpetab otsingu kohe, kui leiab sobiva vaste.